In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
%matplotlib inline

path_to_data = '/home/job/competitions/ML_Boot_Camp_game/data/'

train = pd.read_csv(path_to_data+'x_train.csv', sep = ';')
test = pd.read_csv(path_to_data+'x_test.csv', sep = ';')
y = pd.read_csv(path_to_data+'y_train.csv', header = None)[0]

In [ ]:
train = train.drop('numberOfAttemptedLevels', axis = 1)
test = test.drop('numberOfAttemptedLevels', axis = 1)

train['NumOfAttempts_div'] = train['attemptsOnTheHighestLevel']/train['totalNumOfAttempts']
test['NumOfAttempts_div'] = test['attemptsOnTheHighestLevel']/test['totalNumOfAttempts']

# train['NumOfAttempts_total_score'] = train['totalScore']/train['maxPlayerLevel']
# test['NumOfAttempts_total_score'] = test['totalScore']/test['maxPlayerLevel']

train['NumOfAttempts_total_score'] = train['totalScore']/train['numberOfAttemptedLevels']
test['NumOfAttempts_total_score'] = test['totalScore']/test['maxPlayerLevel']

In [ ]:
cols_to_del = ['maxPlayerLevel', 'totalNumOfAttempts',
               'numberOfBoostersUsed', 'fractionOfUsefullBoosters', 'totalScore',
               'totalBonusScore', 'totalStarsCount']

for col in cols_to_del:
    train[col+'_daily'] = train[col]/train['numberOfDaysActuallyPlayed']

In [ ]:
params = {
    'eta':0.01,
    'booster': 'gbtree',
    'nthread' : 6,
    'objective' : 'binary:logistic',
    'eval_metric' : 'logloss'
}

X = xgb.DMatrix(train, label = y)

cv_res = xgb.cv(params, X, num_boost_round=10000, nfold=5, stratified=True, early_stopping_rounds=50)

In [ ]:
cv_res[cv_res['test-logloss-mean'] == cv_res['test-logloss-mean'].min()]

In [ ]:
gbm = xgb.train(params, X, num_boost_round=1000)

In [ ]:
xgb.plot_importance(gbm)